In [43]:
import datetime, pickle, io
import pandas as pd
import numpy as np
import seaborn as sns
import folium
import folium.plugins as plugins
from tqdm import tqdm
from PIL import Image
from copy import deepcopy
from itertools import product

In [2]:
def parse_df_datetime(df):
    datetime = pd.to_datetime(df.date)
    hours = pd.to_timedelta(df.lengthofday, unit="hours")

    df["parsed_date"] = datetime + hours
    return df

def preprocess_df(df):
    df = parse_df_datetime(df)
    return df

In [3]:
df1 = pd.read_csv("data/continental.csv").iloc[:, 1:]
df1.head()

,date,cases,country,qry,lat,long,dayofyear,year,lengthofday,delta,delta2
0,2020-02-19,1.0,mexico,lat==19.5600234 & long==-99.1312722,19.560023,-99.131272,50,2020,12.274737,0.015820,0.000164
1,2020-02-22,1.0,mexico,lat==19.359004 & long==-99.092623,19.359004,-99.092623,53,2020,12.328092,0.016097,0.000151
2,2020-02-22,1.0,mexico,lat==20.093813 & long==-98.7923761142857,20.093813,-98.792376,53,2020,12.309989,0.016798,0.000156
3,2020-02-23,1.0,mexico,lat==19.482945 & long==-99.113471,19.482945,-99.113471,54,2020,12.341403,0.016363,0.000148
4,2020-02-25,1.0,mexico,lat==16.70864529166667 & long==-93.13649273214284,16.708645,-93.136493,56,2020,12.434892,0.013997,0.000123


In [36]:
df1.shape

(6164889, 12)

In [34]:
(df1.lat == 0).sum(), (df1.long == 0).sum()

(90828, 90828)

In [4]:
df1 = preprocess_df(df1)
# df2 = preprocess_df(df2)
df = df1

# df = pd.concat([df1, df2], axis=0)
df = df.sort_values(by="parsed_date")
df.head()

,date,cases,country,qry,lat,long,dayofyear,year,lengthofday,delta,delta2,parsed_date
5621344,2020-01-01,1.0,argentina,lat==-25.77011895 & long==-61.454868152067206,-25.770119,-61.454868,1,2020,14.586790,-0.007526,-0.000598,2020-01-01 14:35:12.443033796
5621345,2020-01-01,1.0,argentina,lat==-26.61075 & long==-60.2441794,-26.610750,-60.244179,1,2020,14.657257,-0.007816,-0.000621,2020-01-01 14:39:26.126206221
5621346,2020-01-01,1.0,argentina,lat==-27.814298 & long==-64.3006249,-27.814298,-64.300625,1,2020,14.760797,-0.008244,-0.000655,2020-01-01 14:45:38.870751405
5621347,2020-01-01,1.0,argentina,lat==-31.5333655 & long==-68.53848791247032,-31.533365,-68.538488,1,2020,15.103104,-0.009679,-0.000768,2020-01-01 15:06:11.173177260
5621348,2020-01-01,1.0,argentina,lat==-33.3277089 & long==-60.2165909,-33.327709,-60.216591,1,2020,15.282276,-0.010445,-0.000829,2020-01-01 15:16:56.193065871


In [ ]:
# df = df[["parsed_date", "country", "lat", "long"]]

In [44]:
df["parsed_date"].info()

<class 'pandas.core.series.Series'>
Index: 6164889 entries, 5621344 to 145468
Series name: parsed_date
Non-Null Count    Dtype         
--------------    -----         
6164884 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 94.1 MB


In [5]:
df["parsed_date2"] = df["parsed_date"].dt.floor(freq="S")

In [6]:
df.head()

,date,cases,country,qry,lat,long,dayofyear,year,lengthofday,delta,delta2,parsed_date,parsed_date2
5621344,2020-01-01,1.0,argentina,lat==-25.77011895 & long==-61.454868152067206,-25.770119,-61.454868,1,2020,14.586790,-0.007526,-0.000598,2020-01-01 14:35:12.443033796,2020-01-01 14:35:12
5621345,2020-01-01,1.0,argentina,lat==-26.61075 & long==-60.2441794,-26.610750,-60.244179,1,2020,14.657257,-0.007816,-0.000621,2020-01-01 14:39:26.126206221,2020-01-01 14:39:26
5621346,2020-01-01,1.0,argentina,lat==-27.814298 & long==-64.3006249,-27.814298,-64.300625,1,2020,14.760797,-0.008244,-0.000655,2020-01-01 14:45:38.870751405,2020-01-01 14:45:38
5621347,2020-01-01,1.0,argentina,lat==-31.5333655 & long==-68.53848791247032,-31.533365,-68.538488,1,2020,15.103104,-0.009679,-0.000768,2020-01-01 15:06:11.173177260,2020-01-01 15:06:11
5621348,2020-01-01,1.0,argentina,lat==-33.3277089 & long==-60.2165909,-33.327709,-60.216591,1,2020,15.282276,-0.010445,-0.000829,2020-01-01 15:16:56.193065871,2020-01-01 15:16:56


In [26]:
df.shape

(6164889, 13)

In [37]:
df = df.dropna()
df.shape

(6164884, 13)

In [9]:
df["parsed_date2"].min(), df["parsed_date2"].max()

(Timestamp('2020-01-01 14:35:12'), Timestamp('2022-05-20 20:03:36'))

In [22]:
s = datetime.datetime(year=2020, month=7, day=1)
e = datetime.datetime(year=2020, month=8, day=1)
subdf = df[(df["parsed_date2"] >= s) & (df["parsed_date2"] < e)]
print(subdf.shape)

(319690, 13)


In [23]:
subdf.head()

,date,cases,country,qry,lat,long,dayofyear,year,lengthofday,delta,delta2,parsed_date,parsed_date2
5628211,2020-07-01,5.0,argentina,lat==-54.8069332 & long==-68.3073246,-54.806933,-68.307325,183,2020,8.785314,0.014437,0.001578,2020-07-01 08:47:07.131341997,2020-07-01 08:47:07
5796783,2020-07-01,9.0,chile,lat==-53.3527518 & long==-71.5547782,-53.352752,-71.554778,183,2020,8.991017,0.013514,0.001478,2020-07-01 08:59:27.659897876,2020-07-01 08:59:27
5628210,2020-07-01,1.0,argentina,lat==-50.29710505 & long==-72.42061202908346,-50.297105,-72.420612,183,2020,9.377804,0.011845,0.001298,2020-07-01 09:22:40.095306318,2020-07-01 09:22:40
5795979,2020-07-01,0.0,chile,lat==-46.1434629 & long==-74.21278862501612,-46.143463,-74.212789,183,2020,9.827883,0.010003,0.001097,2020-07-01 09:49:40.377315432,2020-07-01 09:49:40
5628209,2020-07-01,10.0,argentina,lat==-42.78708455 & long==-65.07359848469927,-42.787085,-65.073598,183,2020,10.144556,0.008764,0.000962,2020-07-01 10:08:40.399953892,2020-07-01 10:08:40


In [10]:
df[df["parsed_date2"] < datetime.datetime(year=2020, month=2, day=1)].shape

(33809, 13)

In [40]:
df_wo_zero_coord = df[(df.lat != 0) & (df.long != 0)]
df_wo_zero_coord = df_wo_zero_coord.drop_duplicates()
df_wo_zero_coord.shape

(6072174, 13)

In [ ]:
def generate_streaming_data(df):
    year = [2020, 2021]
    month = [1, 7]
    for y, m in product(year, month):
        s = datetime.datetime(year=y, month=m, day=1)
        e = datetime.datetime(year=y, month=m+1, day=1)
        subdf = df_wo_zero_coord[(df_wo_zero_coord["parsed_date2"] >= s) & (df_wo_zero_coord["parsed_date2"] < e)]

        with open(f"parsed/covid_american_streaming_{y}-{m}.txt", "w") as f:
            for i in tqdm(range(subdf.shape[0])):
                date, lat, lng = subdf[["parsed_date2", "lat", "long"]].iloc[i]
                f.write(f"create {i} {lat} {lng} {date}\n")

In [46]:
generate_streaming_data(df_wo_zero_coord)

100%|██████████| 156971/156971 [00:59<00:00, 2640.80it/s]


In [3]:
df1.sort_values(by="date").head()

,date,cases,country,qry,lat,long,dayofyear,year,lengthofday,delta,delta2
0,2020-01-01,1.0,argentina,lat==-25.77011895 & long==-61.454868152067206,-25.770119,-61.454868,1,2020,14.586790,-0.007526,-0.000598
14,2020-01-01,1.0,argentina,lat==-43.2991348 & long==-65.1056655,-43.299135,-65.105666,1,2020,16.523822,-0.016179,-0.001277
13,2020-01-01,1.0,argentina,lat==-39.0266276 & long==-67.5754863,-39.026628,-67.575486,1,2020,15.930364,-0.013328,-0.001055
11,2020-01-01,1.0,argentina,lat==-36.77747 & long==-59.8633562,-36.777470,-59.863356,1,2020,15.658296,-0.012093,-0.000958
10,2020-01-01,1.0,argentina,lat==-34.9206797 & long==-57.9537638,-34.920680,-57.953764,1,2020,15.450322,-0.011174,-0.000886


In [5]:
df1 = parse_df_datetime(df1)
df1.head()

,date,cases,country,qry,lat,long,dayofyear,year,lengthofday,delta,delta2,parsed_date
0,2020-02-19,1.0,mexico,lat==19.5600234 & long==-99.1312722,19.560023,-99.131272,50,2020,12.274737,0.015820,0.000164,2020-02-19 12:16:29.051419738
1,2020-02-22,1.0,mexico,lat==19.359004 & long==-99.092623,19.359004,-99.092623,53,2020,12.328092,0.016097,0.000151,2020-02-22 12:19:41.130106431
2,2020-02-22,1.0,mexico,lat==20.093813 & long==-98.7923761142857,20.093813,-98.792376,53,2020,12.309989,0.016798,0.000156,2020-02-22 12:18:35.959031172
3,2020-02-23,1.0,mexico,lat==19.482945 & long==-99.113471,19.482945,-99.113471,54,2020,12.341403,0.016363,0.000148,2020-02-23 12:20:29.051598591
4,2020-02-25,1.0,mexico,lat==16.70864529166667 & long==-93.13649273214284,16.708645,-93.136493,56,2020,12.434892,0.013997,0.000123,2020-02-25 12:26:05.609865332


In [10]:
def custom_resampler(subdf):
    lats = subdf.lat.to_list()
    lngs = subdf.long.to_list()
    return list(zip(lats, lngs))


def split_data_given_time_unit(df, time_unit):
    timedelta = pd.Timedelta(**time_unit)

    resampled = df.resample(rule=timedelta, origin="start_day", on="parsed_date", label="right")
    resampled_df = resampled.apply(custom_resampler)

    return resampled_df

In [ ]:
time_unit = {
    "weeks": 0,
    "days": 0,
    "hours": 1,
    "minutes": 0,
    "seconds": 0,
}

resampled_df = split_data_given_time_unit(df1, time_unit)

In [8]:
resampled_df.head()

parsed_date
2020-01-01 15:00:00    [(-25.77011895, -61.454868152067206), (-26.610...
2020-01-01 16:00:00    [(-31.5333655, -68.53848791247032), (-33.32770...
2020-01-01 17:00:00                         [(-43.2991348, -65.1056655)]
2020-01-01 18:00:00                                                   []
2020-01-01 19:00:00                                                   []
dtype: object

In [9]:
resampled_df.shape

(20887,)

In [ ]:
mean_lat = df1.lat.mean()
mean_long = df1.long.mean()
m = folium.Map(location=[mean_lat, mean_long], zoom_start=2.5)

features = []

for i in tqdm(range(resampled_df.shape[0])):
# for i in range(300):
    if len(resampled_df.iloc[i]) > 0:
        time = str(resampled_df.index[i]).replace(" ", "T")
        features.extend([
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [lng, lat],
                },
                "properties": {
                    "time": time,
                    "icon": "marker",
                },
            }
            for lat, lng in resampled_df.iloc[i]
        ])
        # print(len(resampled_df.iloc[i]))

    if i == 1000: break


# plugins.TimestampedGeoJson(
#     {
#         "type": "FeatureCollection",
#         "features": features,
#     },
#     period="P1D",
#     # period="PT1H",
#     add_last_point=True,
# ).add_to(m)

# # m.save("vis/vis.html")
# m

In [ ]:
frames = m._to_png()
type(frames)

In [8]:
df.head()

,date,cases,country,qry,lat,long,dayofyear,year,lengthofday,delta,delta2,parsed_date,parsed_date2
5621344,2020-01-01,1.0,argentina,lat==-25.77011895 & long==-61.454868152067206,-25.770119,-61.454868,1,2020,14.586790,-0.007526,-0.000598,2020-01-01 14:35:12.443033796,2020-01-01 14:35:12
5621345,2020-01-01,1.0,argentina,lat==-26.61075 & long==-60.2441794,-26.610750,-60.244179,1,2020,14.657257,-0.007816,-0.000621,2020-01-01 14:39:26.126206221,2020-01-01 14:39:26
5621346,2020-01-01,1.0,argentina,lat==-27.814298 & long==-64.3006249,-27.814298,-64.300625,1,2020,14.760797,-0.008244,-0.000655,2020-01-01 14:45:38.870751405,2020-01-01 14:45:38
5621347,2020-01-01,1.0,argentina,lat==-31.5333655 & long==-68.53848791247032,-31.533365,-68.538488,1,2020,15.103104,-0.009679,-0.000768,2020-01-01 15:06:11.173177260,2020-01-01 15:06:11
5621348,2020-01-01,1.0,argentina,lat==-33.3277089 & long==-60.2165909,-33.327709,-60.216591,1,2020,15.282276,-0.010445,-0.000829,2020-01-01 15:16:56.193065871,2020-01-01 15:16:56


In [12]:
time_unit = {
    "weeks": 1,
    "days": 0,
    "hours": 0,
    "minutes": 0,
    "seconds": 0,
}

s = datetime.datetime(year=2020, month=1, day=1)
e = datetime.datetime(year=2020, month=2, day=1)

resampled_df = split_data_given_time_unit(df[(df["parsed_date2"] >= s) & (df["parsed_date2"] < e)], time_unit)

In [15]:
resampled_df

parsed_date
2020-01-08    [(-25.77011895, -61.454868152067206), (-26.610...
2020-01-15    [(-34.0672039, -60.1029947), (-34.4471919, -58...
2020-01-22    [(-32.2182804, -58.1356171), (-34.4471919, -58...
2020-01-29    [(69.31479216, -153.4836093), (67.04919196, -1...
2020-02-05    [(69.31479216, -153.4836093), (67.04919196, -1...
Freq: 7D, dtype: object

In [36]:
mean_lat = df1.lat.mean()
mean_long = df1.long.mean()


for i in range(resampled_df.shape[0]):
    m = folium.Map(location=[mean_lat, mean_long], zoom_start=2.5)
    acc_count = 0
    points = resampled_df.iloc[i]
    print("Length:", len(points))

    for lat, lng in points:
        folium.CircleMarker(
            location=[lat, lng],
            radius=5
        ).add_to(m)
        # folium.Marker(
        #     location=[lat, lng],
        #     icon=folium.Icon(
        #         color="green",
        #         icon_color='white',
        #     )
        # ).add_to(m)

    acc_count += len(resampled_df.iloc[i])

    map_image = m._to_png(delay=5)
    image = Image.open(io.BytesIO(map_image))
    image.save(f"vis/test_2020_week_{i+1}.png")

Length: 87
Length: 49
Length: 40
Length: 23518
Length: 10115


In [ ]:
mean_lat = df1.lat.mean()
mean_long = df1.long.mean()

m = folium.Map(location=[mean_lat, mean_long], zoom_start=2.5)
acc_count = 0
images = []

# resampled_df

for i in range(resampled_df.shape[0]):
    for lat, lng in resampled_df.iloc[i]:
        folium.Marker(
            location=[lat, lng],
            icon=folium.Icon(
                color="green",
                icon_color='white',
            )
        ).add_to(m)

    acc_count += len(resampled_df.iloc[i])

    if i == 5: break

map_image = m._to_png(delay=5)
image = Image.open(io.BytesIO(map_image))
image.save(f"vis/test.png")
m